<a href="https://colab.research.google.com/github/shadialameddin/tensor_decomposition/blob/master/tensor_decomposition_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

http://tensorly.org/stable/auto_examples/index.html#general-examples

https://github.com/tensorly/tensorly


Sparse backend

Kruskal-tensors have been renamed cp_tensors

Matrix-product-state has now been renamed tensor-train


In [ ]:
!pip install -U tensorly --quiet

In [ ]:
import os 

In [ ]:
if not os.path.exists('tensor_decomposition') and not os.path.isfile('TensorDecomp.py'):
    !git clone https://github.com/shadialameddin/tensor_decomposition.git --quiet

In [ ]:
%cd tensor_decomposition

In [1]:
from TensorDecomp import *
import matplotlib.pyplot as plt

list of available decomposition types:

decomp_list = ['svd', 'parafac', 'tucker', 'matrix_product_state', 'NMF','non_negative_parafac', 'clarkson_woodruff_transform']
#### The Error Handles

In [2]:
err_fro  = lambda  x, y : norm(x-y, ord='fro') / norm(x, ord='fro')
err_L1   = lambda  x, y : norm(x-y, ord=1) / norm(x, ord=1)
err_Linf = lambda  x, y : norm(x-y, ord=inf) / norm(x, ord=inf)
err_Spec = lambda  x, y : norm(x-y, ord=2) / norm(x, ord=2)
    
normL = [err_fro, err_L1, err_Linf, err_Spec]

### Scene Setup

In [3]:
ngp, d, N = 8192, 6, 64

tensor = np.random.rand(ngp, d, N)

# two derived matrices of the tensor (unfolding)
mat_repr1_tens = tensor.reshape(-1, N)
mat_repr2_tens = tensor.reshape(ngp,-1)

# a vector with appropriate dimension to perform operations
vec = np.random.rand(N)

# a matrix with appropriate dimensions to perform operations
someMatrix = np.random.rand(ngp, d)
someTensor = np.random.rand(ngp, d, d)

### Some Tensor Operations ###

# tensor x some vector
tensVec = tensor@vec
#tensVec = np.einsum('klm,m->kl',tensor,vec,optimize ='optimal')

# tensor x some matrix
tensMat = np.einsum('ilk,il->k', tensor, someMatrix, optimize='optimal')

# tensor x some other tensor x tensor
tensTenstens = np.einsum('ilk,ilm,imp->kp', tensor, someTensor, tensor, optimize='optimal')

# matrix representation of a tensor x some vector
matRepr1Vec = mat_repr1_tens@vec

# matrix representation of tensor x some matrix
matRepr1Mat = mat_repr1_tens.T@someMatrix.flatten()

# matrix representations of a tensor x some tensor x matrix representations of a tensor
matReprTensMatRepr = mat_repr1_tens.T @ blk_diag(someTensor) @ mat_repr1_tens

### TIME MEASUREMENT OF TENSOR OPERATIONS ###
n = 1000

# time of tensor x some vector
time_tensVec = timeit.timeit( lambda: tensor@vec, number = n)/n

# time of tensor x some matrix
time_tensMat = timeit.timeit(lambda:np.einsum('ilk,il->k', tensor, someMatrix, optimize='optimal'), number = n)/n

# time of tensor x some other tensor x tensor
time_tensTenstens = timeit.timeit( lambda:np.einsum('ilk,ilm,imp->kp', tensor, someTensor, tensor, optimize='optimal'), number = n)/n

# time of matrix representation of a tensor x some vector
time_matRepr1Vec = timeit.timeit( lambda: mat_repr1_tens@vec, number = n)/n

# time of matrix representations of a tensor x some tensor x matrix representations of a tensor
bsr = blk_diag(someTensor)
time_matReprTensMatRepr = timeit.timeit( lambda:mat_repr1_tens.T @ bsr @ mat_repr1_tens, number = n)/n

# time of matrix representation of tensor x some matrix
flattedMat = someMatrix.flatten()
time_matRepr1Mat = timeit.timeit( lambda: mat_repr1_tens.T@flattedMat, number = n)/n

# SVD

In [4]:
svd_rank = 4
u, s, vt = svds(mat_repr1_tens, svd_rank)
print(u.shape,s.shape,vt.shape)

(49152, 4) (4,) (4, 64)


### Decomposition Time

In [5]:
n = 20
svdDecTime = timeit.timeit((lambda: svds(mat_repr1_tens, svd_rank)), number=n)/n
print("Average SVD Decomposition Time:\t", svdDecTime)

Average SVD Decomposition Time:	 0.52239395


### Memory Saving

In [6]:
memSavSvd = (tensor.nbytes - sum(f.nbytes for f in [u,s,vt])) / tensor.nbytes
print(f"Memory saving with SVD using {svd_rank} Rank:\t", memSavSvd)

Memory saving with SVD using 4 Rank:	 0.9374173482259115


### Decomposition Error

In [7]:
mat_repr1_recons = u @ np.diag(s) @ vt
svdFro, svdL1, svdLinf, svdSpec = [error(mat_repr1_tens, mat_repr1_recons) for error in normL ]

print("Frobenius Norm of the SVD Decomposition Error:\t", svdFro)
print("L1 Norm of the SVD Decomposition Error:\t\t", svdL1)
print("L infinity Norm of the SVD Decomposition Error:\t", svdLinf)
print("Spectral Norm of the SVD Decomposition Error:\t", svdSpec)

Frobenius Norm of the SVD Decomposition Error:	 0.48317893383389954
L1 Norm of the SVD Decomposition Error:		 0.49120537113416163
L infinity Norm of the SVD Decomposition Error:	 0.4682183906457543
Spectral Norm of the SVD Decomposition Error:	 0.07392111402989492


### Error in Decomposed Tensor Operations

In [8]:
# svd of matrix representation of tensor x some vector
svdMatReprVec = u @ (np.diag(s) @ (vt @ vec))

# svd of matrix representation of tensor x some matrix
svdMatReprMat = vt.T @ np.diag(s) @ ( u.T @ someMatrix.flatten())

# svd of matrix representation of tensor x some Tensor x svd of matrix representation of tensor x some
svdMatReprTenssvdMatRepr = vt.T @ np.diag(s) @ ( u.T @ bsr @ u) @ np.diag(s) @ vt

svdDecVec_fro, svdDecVec_L1, svdDecVec_Linf, svdDecVec_L2 = [norm(tensVec,svdMatReprVec.reshape(tensVec.shape)) for norm in normL]
print(svdDecVec_fro, svdDecVec_L1, svdDecVec_Linf, svdDecVec_L2)

tensMat_svdMatReprMat_L2 = err_Spec(tensMat,svdMatReprMat)
print(tensMat_svdMatReprMat_L2)

svdMatReprTenssvdMatRepr_tensTenstens_fro, svdMatReprTenssvdMatRepr_tensTenstens_L1, svdMatReprTenssvdMatRepr_tensTenstens_Linf, svdMatReprTenssvdMatRepr_tensTenstens_L2 = [norm(tensTenstens,svdMatReprTenssvdMatRepr) for norm in normL]
print(svdMatReprTenssvdMatRepr_tensTenstens_fro, svdMatReprTenssvdMatRepr_tensTenstens_L1, svdMatReprTenssvdMatRepr_tensTenstens_Linf, svdMatReprTenssvdMatRepr_tensTenstens_L2)

0.0482092088928584 0.039264702982910495 0.08459878416819341 0.020122865599680937
0.0015769524501791743
0.0068008448658964035 0.0030542397851032106 0.003431927547579893 0.0011054443786768628


### Decomposition Speed Up Measures in Tensor Operations

In [9]:
# time of svd of matrix representation of tensor x some vector
time_svdMatReprVec = timeit.timeit( lambda:u @ (np.diag(s) @ (vt @ vec)), number = n)/n

# time of svd of matrix representation of tensor x some matrix
time_svdMatReprMat = timeit.timeit( lambda:vt.T @ np.diag(s) @ ( u.T @ flattedMat), number = n)/n

# time of svd of matrix representation of tensor x some Tensor x svd of matrix representation of tensor x some
time_svdMatReprTenssvdMatRepr = timeit.timeit( lambda:vt.T @ np.diag(s) @ ( u.T @ bsr @ u) @ np.diag(s) @ vt, number = n)/n

# Speed up in operations

### 
speedup_tensVec_matReprVec = time_tensVec / time_matRepr1Vec
speedup_matRepr1Vec_svdMatReprVec =  time_matRepr1Vec / time_svdMatReprVec
speedup_tensVec_svdMatReprVec = time_tensVec / time_svdMatReprVec

###
speedup_tensMat_matRepr1Mat = time_tensMat / time_matRepr1Mat
speedup_matRepr1Mat_svdMatReprMat = time_matRepr1Mat / time_svdMatReprMat
speedup_tensMat_svdMatReprMat = time_tensMat / time_svdMatReprMat

###
speedup_tenstenstens_matReprTensMatRepr = time_tensTenstens / time_matReprTensMatRepr
speedup_matReprTensMatRepr_svdMatReprTenssvdMatRepr = time_matReprTensMatRepr / time_svdMatReprTenssvdMatRepr 
speedup_tenstenstens_svdMatReprTenssvdMatRepr = time_tensTenstens / time_svdMatReprTenssvdMatRepr 

print(speedup_tensVec_matReprVec, speedup_matRepr1Vec_svdMatReprVec, speedup_tensVec_svdMatReprVec)
print(speedup_tensMat_matRepr1Mat, speedup_matRepr1Mat_svdMatReprMat, speedup_tensMat_svdMatReprMat)
print(speedup_tenstenstens_matReprTensMatRepr, speedup_matReprTensMatRepr_svdMatReprTenssvdMatRepr, speedup_tenstenstens_svdMatReprTenssvdMatRepr )

1.0617518057165605 7.334928951567589 7.787874059129565
1.0185782301640012 15.86770385512882 16.162497709523613
1.3425908641662154 4.838773830499043 6.496493538594578


# Tucker Decomposition

In [10]:
rank_tucker = (4,4,4)
core, factors = tucker(tensor, rank_tucker)
print("Core Dimension",core.shape)
print("Factors Dimensions", [f.shape for f in factors])

Core Dimension (4, 4, 4)
Factors Dimensions [(8192, 4), (6, 4), (64, 4)]


### Decomposition Time

In [11]:
n = 10
tuckerDecTime = timeit.timeit((lambda: tucker(tensor, rank_tucker)), number=n)/n
print("Average Tucker Decomposition Time:\t", tuckerDecTime)

Average Tucker Decomposition Time:	 0.39413333000000106


### Memory Saving

In [12]:
memSavTucker = (tensor.nbytes - (core.nbytes + sum(f.nbytes for f in factors))) / tensor.nbytes
print(f"Memory saving with Tucker Decomposition using {rank_tucker} Rank:\t", memSavTucker)

Memory saving with Tucker Decomposition using (4, 4, 4) Rank:	 0.9894739786783854


### Decomposition Error

In [13]:
tensor_approx_tucker = np.einsum('lmn,il,jm,kn->ijk',core,factors[0],factors[1],factors[2])
tuckerFro, tuckerL1, tuckerLinf, tuckerSpec = [error(tensor.reshape(-1,N), tensor_approx_tucker.reshape(-1,N)) for error in normL ]

print("Frobenius Norm of the Tucker Decomposition Error:\t", tuckerFro)
print("L1 Norm of the Tucker Decomposition Error:\t\t", tuckerL1)
print("L infinity Norm of the Tucker Decomposition Error:\t", tuckerLinf)
print("Spectral Norm of the Tucker Decomposition Error:\t", tuckerSpec)

Frobenius Norm of the Tucker Decomposition Error:	 0.4968996454415876
L1 Norm of the Tucker Decomposition Error:		 0.4960061839926649
L infinity Norm of the Tucker Decomposition Error:	 0.4881376225901378
Spectral Norm of the Tucker Decomposition Error:	 0.07392600894017429


### Decomposed Tensor Operations

In [14]:
# decomposed tensor @ vec
tuckerTens_Vec = np.einsum('klm,pk,rl,s->pr', core, factors[0], factors[1], factors[2].T @ vec)

#parTensorVec = np.einsum('lmn,il,jm,kn->ijk',core,factors[0],factors[1],factors[2])

# decomposed tensor @ mat
tuckerTens_Mat = np.einsum('klm,pk,pl,rm->r', core, (someMatrix.T@factors[0]), factors[1], factors[2])

# decomposed tensor @ someTensor @ decomposed tensor
#tuckerTens_tensor_tuckerTens = np.einsum( optimize='optimal')

### Error between decomposed tensor - matrix and tensor-matrix operations

In [15]:
tuckerTensVec_fro, tuckerTensVec_L1, tuckerTensVec_Linf, tuckerTensVec_L2 = [error(tensVec, tuckerTens_Vec ) for error in normL]

print(tuckerTensVec_fro, tuckerTensVec_L1, tuckerTensVec_Linf, tuckerTensVec_L2)

tuckerTensMat_L2 = err_Spec(tensMat, tuckerTens_Mat) 

print(tuckerTensMat_L2)

#parTensTens_fro, parTensTens_L1, parTensTens_Linf, parTensTens_L2 = [error(tensTenstens, parTensTens) for error in normL]

0.09815107346626657 0.10257589389980794 0.16155584710257365 0.05665529139231639
0.0015817976975723556


### Decomposition Speed Up Measures in Tensor Operations

In [16]:
n= 100

# time of tucker tensor x some vector
time_tuckerTens_Vec = timeit.timeit(lambda:np.einsum('klm,pk,rl,s->pr', core, factors[0], factors[1], factors[2].T @ vec, optimize ='optimal'), number = n)/n

# time of tucker tensor x some matrix
time_tuckerTens_Mat = timeit.timeit( lambda:np.einsum('klm,pk,pl,rm->r', core, (someMatrix.T@factors[0]), factors[1], factors[2],optimize ='optimal'), number = n)/n

# time of tucker tensor x some Tensor x tucker tensor
#time_parTens_tensor_parTens = timeit.timeit( lambda:vt.T @ np.diag(s) @ ( u.T @ blk_diag(someTensor) @ u) @ np.diag(s) @ vt, number = n)/n

In [17]:
speedUP_tensVec_tuckerTensVec = time_tensVec / time_tuckerTens_Vec
speedUP_tensMat_tuckerTensMat = time_tensMat / time_tuckerTens_Mat
#speedUP_tensTensortens_parTensTensorParTens = time_tensTenstens / time_time_parTens_tensor_parTens

print(speedUP_tensVec_tuckerTensVec,speedUP_tensMat_tuckerTensMat)

5.636170971281331 4.364584637813302


# Parafac Decomposition

### Decomposition Time

### Memory Saving

### Decomposition Error

### Decomposed Tensor Operations

### Error between decomposed tensor - matrix and tensor-matrix operations

### Decomposition Speed Up Measures in Tensor Operations

In [ ]:
errTucker = errList(tensor, tucker, vecR, vecL, matR, matL, normL, rank=[128,64,6])
timeTucker = tensor.decomp_time
memSavTucker = tensor.memSaving

In [ ]:
errParafac = errList(tensor, parafac, vecR, vecL, matR, matL, normL, rank=384)
timeParafac = tensor.decomp_time
memSavParafac = tensor.memSaving

In [ ]:
errTT  = errList(tensor, matrix_product_state, vecR, vecL, matR, matL, normL, rank=[1,128,128,1])
timeTT = tensor.decomp_time
memSavTT = tensor.memSaving

In [ ]:
errNNP  = errList(tensor, non_negative_parafac, vecR, vecL, matR, matL, normL, rank= 384)
timeNNP = tensor.decomp_time
memSavNNP = tensor.memSaving

In [ ]:
%matplotlib inline

xAxis = ['SVD', 'Tucker', 'Parafac', 'Tensor Train', 'Non-negative Parafac']
y0Axis = [errSvd[0][0], errTucker[0][0], errParafac[0][0], errTT[0][0], errNNP[0][0]]
y1Axis = [timeSvd, timeTucker, timeParafac, timeTT, timeNNP]
y2Axis = [memSavSvd, memSavTucker, memSavParafac, memSavTT, memSavNNP]
colors = ["crimson","deepskyblue", "yellowgreen"]

fig, ax = plt.subplots(3, 1, figsize=(20,16), sharex=True)
for i in range(3):
    ax[i].bar(xAxis, eval('y'+str(i)+'Axis'), color=colors[i])
    ax[i].yaxis.grid(color='b', alpha=0.5, linestyle='dashed', linewidth=0.5)
    ax[i].set_axisbelow(True)
    ax[i].set_ylim(0, np.ceil(max(eval('y'+str(i)+'Axis'))) + np.mean(eval('y'+str(i)+'Axis'))) if max(eval('y'+str(i)+'Axis')) > 0 else ax[i].set_ylim(0, np.floor(min(eval('y'+str(i)+'Axis')))-5)
    ax[i].tick_params(axis='both', which='major', labelsize=16)
    for h, v in enumerate(eval('y'+str(i)+'Axis')):
        ax[i].text(h-.10, v+.0125, str(np.round(v,4)), color='black', size = 16, weight = "bold")

plt.xlabel("Decomposition Types", fontsize=24)
ax[0].set_ylabel("Relative Error [-]", fontsize=18)
ax[1].set_ylabel("Time [sec]", fontsize=18)
ax[2].set_ylabel("Memory Saving [-]", fontsize=18)

In [ ]:
operList = [lambda:tensor.tensor@vecR, lambda:vecL@tensor.tensor, lambda:tensor.tensor@matR, lambda:matL@tensor.tensor, lambda:vecL@tensor.tensor@vecR, 
lambda:matL@tensor.tensor@matR]

tensorOperationTimes = tensOpTim(operList)

In [ ]:
from matplotlib import ticker
uniSlightBlue = '#00BEFF'
fig, ax = plt.subplots(1, 1, figsize=(20,8))
ax.bar(operList, tensorOperationTimes, color = uniSlightBlue)
ax.yaxis.grid(color='b', alpha=0.5, linestyle='dashed', linewidth=0.5)
plt.xticks(rotation=22.5)
ax.set_axisbelow(True)
ax.set_ylabel("Elapsed Time [sec]", fontsize=24)
ax.tick_params(axis='both', which='major', labelsize=16)

formatter = ticker.ScalarFormatter(useMathText=True)
formatter.set_scientific(True) 
formatter.set_powerlimits((-1,1)) 
ax.yaxis.set_major_formatter(formatter) 

In [ ]:
#TODO #1
# Speed comparion is not implemented. No function definition is defined for decomposed_tensor @ Vec multiplication.

In [ ]:
#TODO #2
# For every decomposition type -> Line Plot: decomposition error, speedup time in tensor operations vs decomposed tensor operations.

In [ ]:
# DONE! (11.07.21)
# For every decomposition type -> Bar plot: Decomposition error, decomposition time, memory saving